In [1]:
%load_ext autoreload
%autoreload 2

import os
import fnmatch
from pipeline.lda import LDABuilder
from pipeline.utils import *
import numpy as np
import pandas as pd

In [2]:
ldab = LDABuilder('all_the_news')
paths = ldab.paths

In [ ]:
dfs = []
for fp in [paths.data_file('articles' + str(i) + '.csv') for i in [3,1,2]]:
    print(fp)
    dfs.append( pd.read_csv(fp))
        
# dfs

In [ ]:
df = pd.concat(dfs).reset_index(drop=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
# df['year'].astype(int)
# df['month'].astype(int)
df.info()

In [ ]:
for i,parsed_doc in enumerate(read_doc_by_line(paths.trigram_corpus_filepath)):

    if i < 5 or (i >200000 and i < 200000+6) or i > len(df) - 7:
        print(parsed_doc[:50])
        print(df.loc[i].content[:50])

In [ ]:
import re
def partial_text(text):
    text = re.sub("['\" ]{2,}", " ", text)
    return prep_whitespace(text)[:500] + '...'

In [ ]:
df['partial_content'] = df['content'].map(lambda x: partial_text(x))
df.drop(columns=['content'], inplace=True)
df.to_csv(paths.corpus_meta_data)
df.info()

In [ ]:
df.head()

Note this is bad

In [ ]:
df[df.year.isin([np.nan, np.inf, -np.inf])]

In [18]:
%%time
matrix = ldab.get_corpus_topics_matrix(recalculate=True)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 11 µs
Loading LDA model (n_topics=50)...
Loading trigram dict...


In [19]:
%%time
matrix = ldab.get_corpus_topics_matrix(75, recalculate=True)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 10 µs
Loading LDA model (n_topics=75)...
Loading trigram dict...


In [ ]:
%%time
matrix = ldab.get_corpus_topics_matrix(100, recalculate=True)

Loading LDA model (n_topics=100)...
Loading trigram dict...
No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and sh

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 

No topics found, which is strange and should not happen for a real news article.
No good article: 



In [23]:
from pipeline.preprocessing import Preprocessor
prep = Preprocessor('all_the_news', preload_models=True)
meta_data = pd.read_csv(ldab.paths.corpus_meta_data)

load spacy model
['tagger', 'sbd']
Loading bi-gram model...
Done!
Loading tri-gram model...
Done!


In [29]:
from random import *

index = randint(0,len(df)-1)
text = df.content[index]
lda = ldab.get_lda_model(100)
print(df.loc[index])
print(text[:1500])

Loading LDA model (n_topics=100)...
id                                                         72307
title          Watch National Geographic’s stunning climate-c...
publication                                     Business Insider
author                                                       NaN
date                                                  2016-10-31
year                                                        2016
month                                                         10
url                                                          NaN
content        ’  ’ ’   You have to admit, it doesn’’t sound ...
Name: 91680, dtype: object
’  ’ ’   You have to admit, it doesn’’t sound like     TV: a National Geographic Channel documentary about climate   change starring Leonardo DiCaprio. Mixing together the   science of our damaged planet and the guy who fought the bear in   the movie ”The Revenant” seems like a bit of an odd choice. ’ ’   But the combination works. DiCaprio steps int

In [47]:
parsed_doc = prep.process_doc(' asdkjf')
corpus_dict = ldab.get_corpus_dict()
bow = corpus_dict.doc2bow(parsed_doc)
doc_topics = lda[bow]
topic_vec = ldab.create_topic_vec(lda.num_topics, doc_topics)
topic_vec

Loading trigram dict...
No topics found, which is strange and should not happen for a real news article.


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [95]:
n = 2
sorted_top = np.argpartition(topic_vec, -n)[-n:][::-1]

[print(lda.print_topic(i) + '\n') for i in sorted_top]
1

0.022*"trump" + 0.007*"u._s." + 0.007*"president" + 0.006*"country" + 0.005*"military" + 0.004*"united_states" + 0.004*"administration" + 0.004*"official" + 0.004*"obama" + 0.003*"american"

0.039*"trump" + 0.012*"clinton" + 0.009*"president" + 0.008*"campaign" + 0.006*"donald_trump" + 0.005*"mr." + 0.005*"hillary_clinton" + 0.004*"think" + 0.004*"ask" + 0.004*"election"



1